In [ ]:
from database_wrapper import DatabaseWrapper

db = DatabaseWrapper("listings.db")
df = db.get_df()
db.close_conn()
df

In [ ]:
# print unique disposition values
print(df["disposition"].unique())
dispositions = df["disposition"].unique()

In [ ]:
from sreality_scraper.sreality.spiders.sreality_spider import SrealityUrlBuilder

In [ ]:
# for each unique disposition value try running the map_category_sub_cb from SrealityUrlBuilder
for disposition in dispositions:
    if not disposition:
        continue
    
    df['disposition'] = df['disposition'].replace(disposition, SrealityUrlBuilder.map_category_sub_cb(disposition))

        

In [ ]:
df['disposition'].unique()

In [ ]:
df['disposition'] = df['disposition'].replace('Garsoniéra', '1+kk') # bezrealitky specific
df['disposition'] = df['disposition'].replace('Ostatní', 'ostatní') # bezrealitky specific
df['disposition'] = df['disposition'].replace('atypicky', 'ostatní') # sreality specific
df['disposition'] = df['disposition'].replace('pokoj', 'ostatní') # sreality specific
df['disposition'] = df['disposition'].replace('6+kk', '6-a-více')
df['disposition'] = df['disposition'].replace('6+1', '6-a-více')
df['disposition'] = df['disposition'].replace('7+kk', '6-a-více')
df['disposition'] = df['disposition'].replace('7+1', '6-a-více')


In [ ]:
df['disposition'].sort_values().unique()

In [ ]:
df

In [ ]:
# drop entries with area bigger than 1000
df = df[df['area'] < 1000]
df['area'].sort_values()
